In [1]:
from TCSPC import *

In [2]:
EGFP = Simulation([0.497,0.503],[2.43,3.07])

In [3]:
# tdata = np.loadtxt('EGFPt.csv')
# ydata = np.loadtxt('EGFPy2.csv')

In [4]:
def trim_rescale_data(tdata,ydata,end =int(0.6*380),rescale = True):
    max_idx = np.argmax(ydata) #index of data point with maximum photon count N(0)
    tdata = tdata[:end-max_idx] #start from t = 0
    ydata = ydata[max_idx:end]  #start from max.
    if rescale == True:
        yerr = ydata/ydata[0]*np.sqrt(1/ydata+1/ydata[0]) #error after scaling
        ydata = ydata/ydata[0] # scale y data such that the beginning is 1 
    else:
        yerr = np.sqrt(ydata)
    weights = 1/yerr #weighted by 1/yerr, yerr is error after scaling ydata
    return tdata,ydata,weights

In [5]:
def exp(t, A, tau):
    return A * np.exp(-t/tau)

In [6]:
tdata,ydata,weights = trim_rescale_data(EGFP.t,EGFP.y2)

In [7]:
def poisson_deviance_residual(observed, expected):
    # Ensure no invalid values
    # valid = (observed > 0) & (expected > 0)
    # residual = np.zeros_like(observed)
    # residual[valid] =  2 *  (observed[valid] * np.log(observed[valid] / expected[valid]) - (observed[valid] - expected[valid]))
    residual=  np.sqrt(abs(2 *  (observed* np.log(observed/ expected) - (observed- expected)))) #residual array
    return residual

def residual(p, t, data):
    v = p.valuesdict()
    generative = v['c'] #constant background
    M = 1
    while f'A{M}' in v:
        generative += exp(t, v[f'A{M}'], v[f'tau{M}'])
        M += 1
    
    return poisson_deviance_residual(data, generative) #lmfit.minimizer minimize the residual array in the sum of squared sense


In [8]:
def initial_params(M,A_guess,tau_guess,rescale = True):
    p = lmfit.Parameters()
    p.add_many(('c', 0, True, 0, 1)) #constant background
    for i in range(1,M+1): #for each component
        p.add_many((f'A{i}', A_guess[i-1], True,0), #amplitude
                   (f'tau{i}', tau_guess[i-1], True, 0)) #lifetime
    if rescale == True:
        p[f'A{M}'].set(expr = f'1 {"".join([f"- A{i}" for i in range(1,M)])}') #fix the amplitude of last component
    return p



### bi-exp decay fit to EGFP

In [35]:
EGFP.n_photon = int(1e4)
EGFP.multi_exp_data()
tdata,ydata,weights = trim_rescale_data(EGFP.t,EGFP.y2,end = int(0.5*380))
p1 = initial_params(2,EGFP.amp,EGFP.tau)
#p1['c'].set(value = 0, vary = False)
mi1 = lmfit.minimize(residual, p1, args=(tdata, ydata), method='powell')

print(lmfit.fit_report(mi1))


[[Fit Statistics]]
    # fitting method   = Powell
    # function evals   = 244
    # data points      = 180
    # variables        = 4
    chi-square         = 0.97722245
    reduced chi-square = 0.00555240
    Akaike info crit   = -930.879606
    Bayesian info crit = -918.107779
##  Warning: uncertainties could not be estimated:
[[Variables]]
    c:     0.00142491 +/-        nan (nan%) (init = 0)
    A1:    0.58982962 +/-        nan (nan%) (init = 0.497)
    tau1:  2.39734468 +/-        nan (nan%) (init = 2.43)
    A2:    0.41017038 +/- 0.00000000 (0.00%) == '1 - A1'
    tau2:  3.10072775 +/- 0.47030090 (15.17%) (init = 3.07)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lmfit/minimizer.py:847: RuntimeWarning: invalid value encountered in sqrt
  par.stderr = np.sqrt(self.result.covar[ivar, ivar])
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lmfit/minimizer.py:854: RuntimeWarning: invalid value encountered in sqrt
  (par.stderr * np.sqrt(self.result.covar[jvar, jvar])))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/uncertainties/core.py:173: RuntimeWarning: invalid value encountered in sqrt
  std_devs = numpy.sqrt(numpy.diag(covariance_mat))


In [34]:
print(lmfit.fit_report(mi1))

[[Fit Statistics]]
    # fitting method   = Powell
    # function evals   = 441
    # data points      = 180
    # variables        = 4
    chi-square         = 0.00958461
    reduced chi-square = 5.4458e-05
    Akaike info crit   = -1763.29961
    Bayesian info crit = -1750.52779
[[Variables]]
    c:     6.3832e-12 +/- 6.8356e-08 (1070882.25%) (init = 0)
    A1:    0.56732075 +/- 0.24709655 (43.56%) (init = 0.497)
    tau1:  2.40917616 +/- 0.16062625 (6.67%) (init = 2.43)
    A2:    0.43267925 +/- 0.24709655 (57.11%) == '1 - A1'
    tau2:  3.19202027 +/- 0.24349024 (7.63%) (init = 3.07)
[[Correlations]] (unreported correlations are < 0.100)
    C(A1, tau2)   = 0.994
    C(A1, tau1)   = 0.992
    C(tau1, tau2) = 0.974


In [10]:
result = mi1
del result.params['A2']

In [15]:
[v.value for v in result.params.values()]

[0.0027020926751966656,
 0.3783589907794347,
 2.721952066350494,
 2.7532366205856342]

In [22]:
info_df,par_df = fit_df([result])


In [23]:
par_df

c  \
0 _val                                                 0.000854   
  init_value                                                  0   
  stderr                                               0.000457   
  correl      {'A0': '-0.734', 'tau0': '-0.701', 'tau1': '-0...   

                                                             A0  \
0 _val                                                  0.50092   
  init_value                                              0.497   
  stderr                                               0.842672   
  correl      {'c': '-0.734', 'tau0': '0.998', 'tau1': '0.998'}   

                                                         tau0  \
0 _val                                               2.471439   
  init_value                                             2.43   
  stderr                                             0.429631   
  correl      {'c': '-0.701', 'A0': '0.998', 'tau1': '0.994'}   

                                                         tau1  
0 _val                                               3.030307  
  init_value                                             3.07  
  stderr                                             0.526496  
  correl      {'c': '-0.769', 'A0': '0.998', 'tau0': '0.994'}

### No rescale

In [29]:
EGFP.n_photon = int(1e4)
EGFP.multi_exp_data()
tdata,ydata,weights = trim_rescale_data(EGFP.t,EGFP.y2,end = 100,rescale = False)
p1 = initial_params(2,[np.max(EGFP.y2*0.48),np.max(EGFP.y2)*0.52],EGFP.tau,rescale = False)
#p1['c'].set(value = 0, vary = False)
mi1 = lmfit.minimize(residual, p1, args=(tdata, ydata),method = 'powell')

print(lmfit.fit_report(mi1))



[[Fit Statistics]]
    # fitting method   = Powell
    # function evals   = 119
    # data points      = 87
    # variables        = 5
    chi-square         = 80.4674607
    reduced chi-square = 0.98131050
    Akaike info crit   = 3.20919493
    Bayesian info crit = 15.5387355
##  Warning: uncertainties could not be estimated:
    c:     at boundary
[[Variables]]
    c:     0.00000000 +/- 5.3809e-09 (inf%) (init = 0)
    A1:    76.9305131 +/-        nan (nan%) (init = 91.2)
    tau1:  2.42745657 +/-        nan (nan%) (init = 2.43)
    A2:    98.7744840 +/-        nan (nan%) (init = 98.8)
    tau2:  3.06269655 +/- 0.47375377 (15.47%) (init = 3.07)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lmfit/minimizer.py:854: RuntimeWarning: invalid value encountered in sqrt
  (par.stderr * np.sqrt(self.result.covar[jvar, jvar])))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lmfit/minimizer.py:847: RuntimeWarning: invalid value encountered in sqrt
  par.stderr = np.sqrt(self.result.covar[ivar, ivar])
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lmfit/printfuncs.py:175: RuntimeWarning: divide by zero encountered in scalar divide
  spercent = f'({abs(par.stderr/par.value):.2%})'


In [26]:
A = [mi1.params[f'A{i}'].value for i in range(1,3)]
A/np.sum(A)

array([0.45733277, 0.54266723])

### mono-exp decay fit to EGFP

In [109]:
p1

name,value,initial value,min,max,vary,expression
c,0.00000000,0,0.00000000,1.00000000,True,
A0,0.49700000,0.497,0.00000000,inf,True,
tau0,2.43000000,2.43,0.00000000,inf,True,
A1,0.50300000,0.503,0.00000000,inf,False,1 - A0
tau1,3.07000000,3.07,0.00000000,inf,True,


In [33]:
EGFP.n_photon = int(1e4)
EGFP.multi_exp_data()
tdata,ydata,weights = trim_rescale_data(EGFP.t,EGFP.y2,end = 100,rescale = False)
p1 = initial_params(3,[np.max(EGFP.y2*0.48),np.max(EGFP.y2)*0.52,1],EGFP.tau+[1],rescale = False)
#p1['c'].set(value = 0, vary = False)
mi1 = lmfit.minimize(residual, p1, args=(tdata, ydata),method = 'powell')

print(lmfit.fit_report(mi1))

[[Fit Statistics]]
    # fitting method   = Powell
    # function evals   = 234
    # data points      = 89
    # variables        = 7
    chi-square         = 70.0320654
    reduced chi-square = 0.85404958
    Akaike info crit   = -7.33180083
    Bayesian info crit = 10.0886538
##  Warning: uncertainties could not be estimated:
    c:     at boundary
[[Variables]]
    c:     2.1705e-14 +/- 8.4829e-07 (3908303340.74%) (init = 0)
    A1:    80.5031335 +/-        nan (nan%) (init = 95.52)
    tau1:  2.43395215 +/-        nan (nan%) (init = 2.43)
    A2:    103.350749 +/-        nan (nan%) (init = 103.48)
    tau2:  3.06611503 +/-        nan (nan%) (init = 3.07)
    A3:    0.38020313 +/- 74.5156834 (19598.91%) (init = 1)
    tau3:  2.66338794 +/- 16.8327766 (632.01%) (init = 1)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lmfit/minimizer.py:854: RuntimeWarning: invalid value encountered in sqrt
  (par.stderr * np.sqrt(self.result.covar[jvar, jvar])))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lmfit/minimizer.py:847: RuntimeWarning: invalid value encountered in sqrt
  par.stderr = np.sqrt(self.result.covar[ivar, ivar])


In [118]:
np.sum(residual(mi1.params,tdata,ydata))

29.50655967458046